#### Import Libraries 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf

#### Import data 

it was noticed during examining the data that there are missing values in the location columns that is diplayed as -1 and (0,0), this will not show as missing values during analysis as it is not a NaN value so we should remove it to better understand our data.

this can be defined during import of data by specifying which data is to be considered missing

In [2]:
#before import define missing values for this data
missing_values = ["n/a", "na", -1, '(0.00000000, 0.00000000)', '(-1.00000000, -1.00000000)']

#import the data
df = pd.read_csv('boston.csv', header=0, encoding = "ISO-8859-1", low_memory=False, na_values = missing_values)

#### check that the data loaded properly:

In [3]:
df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I192082859,724,Auto Theft,AUTO THEFT,E18,519,NaN,2019-10-13 09:28:24,2019,10,Sunday,9,Part One,LINCOLN ST,42.259518,-71.121563,"(42.25951765, -71.12156299)"
1,I192082751,724,Auto Theft,AUTO THEFT,E18,493,NaN,2019-10-12 20:11:26,2019,10,Saturday,20,Part One,METROPOLITAN AVE,42.262092,-71.116710,"(42.26209214, -71.11670964)"
2,I192082680,727,Auto Theft,AUTO THEFT - LEASED/RENTED VEHICLE,D14,794,NaN,2019-10-12 15:12:43,2019,10,Saturday,15,Part One,ALLSTON ST,42.352375,-71.135096,"(42.35237455, -71.13509584)"
3,I192082577,724,Auto Theft,AUTO THEFT,D4,130,NaN,2019-10-12 04:41:52,2019,10,Saturday,4,Part One,SAINT JAMES AVE,42.349476,-71.076402,"(42.34947586, -71.07640150)"
4,I192079582,727,Auto Theft,AUTO THEFT - LEASED/RENTED VEHICLE,A15,47,NaN,2019-10-02 08:08:49,2019,10,Wednesday,8,Part One,N MEAD ST,42.381846,-71.066551,"(42.38184582, -71.06655134)"


#### initial data check 

In [4]:
#get the column names:
column_names = df.columns

#print column names:
for name in column_names:
    print(name)

INCIDENT_NUMBER
OFFENSE_CODE
OFFENSE_CODE_GROUP
OFFENSE_DESCRIPTION
DISTRICT
REPORTING_AREA
SHOOTING
OCCURRED_ON_DATE
YEAR
MONTH
DAY_OF_WEEK
HOUR
UCR_PART
STREET
Lat
Long
Location


In [5]:
#check data types of each column
df.dtypes

INCIDENT_NUMBER         object
OFFENSE_CODE             int64
OFFENSE_CODE_GROUP      object
OFFENSE_DESCRIPTION     object
DISTRICT                object
REPORTING_AREA          object
SHOOTING                object
OCCURRED_ON_DATE        object
YEAR                     int64
MONTH                    int64
DAY_OF_WEEK             object
HOUR                     int64
UCR_PART                object
STREET                  object
Lat                    float64
Long                   float64
Location                object
dtype: object

In [6]:
#get count of dtypes
df.dtypes.value_counts()

object     11
int64       4
float64     2
dtype: int64

In [7]:
#check if each column has unique values only:
for name in column_names:
    print('{} is unique: {}'.format(name, df[name].is_unique))

INCIDENT_NUMBER is unique: False
OFFENSE_CODE is unique: False
OFFENSE_CODE_GROUP is unique: False
OFFENSE_DESCRIPTION is unique: False
DISTRICT is unique: False
REPORTING_AREA is unique: False
SHOOTING is unique: False
OCCURRED_ON_DATE is unique: False
YEAR is unique: False
MONTH is unique: False
DAY_OF_WEEK is unique: False
HOUR is unique: False
UCR_PART is unique: False
STREET is unique: False
Lat is unique: False
Long is unique: False
Location is unique: False


check if there is an index associated with the dataframe


In [8]:
# Check the index values
df.index.values


array([     0,      1,      2, ..., 426837, 426838, 426839], dtype=int64)

In [9]:
# Check if a certain (random) index exists
100 in df.index.values

True

at this point the data has been quickly checked:
    1. we know the data types 
    2. we know if the columns have unique values only 
    3. we know that it has an index so we can do merge and join later if needed

## Data Cleaning:

### Find the missing values:

In [10]:
# display how many missing values per column
df.isnull().sum()

INCIDENT_NUMBER             0
OFFENSE_CODE                0
OFFENSE_CODE_GROUP          0
OFFENSE_DESCRIPTION         0
DISTRICT                 2169
REPORTING_AREA              0
SHOOTING               425093
OCCURRED_ON_DATE            0
YEAR                        0
MONTH                       0
DAY_OF_WEEK                 0
HOUR                        0
UCR_PART                  110
STREET                  12391
Lat                     28131
Long                    28131
Location                28131
dtype: int64

### Missing Data 

it can be noticed that the majority of the missing data is in the <b>SHOOTING</b> column, however this doesnt mean that the shooting information is missing in our data as the shooting column contain a value only for incidents where shooting took place. to fix this we will replace the null values with a (N) in the shooting column. 

In [11]:
df['SHOOTING'].fillna('N', inplace=True)

In [12]:
# display how many missing values per column after filling shooting with (N)
df.isnull().sum()

INCIDENT_NUMBER            0
OFFENSE_CODE               0
OFFENSE_CODE_GROUP         0
OFFENSE_DESCRIPTION        0
DISTRICT                2169
REPORTING_AREA             0
SHOOTING                   0
OCCURRED_ON_DATE           0
YEAR                       0
MONTH                      0
DAY_OF_WEEK                0
HOUR                       0
UCR_PART                 110
STREET                 12391
Lat                    28131
Long                   28131
Location               28131
dtype: int64

### Missing Location Information

we can notice that we have the same number of missing values for <b>Lat</b>, <b>Long</b>, and <b>Location</b> <u>(28,131 rows)</u>

we also have (12,391) missing values in STREET, the following will try to check if there is overlap between the missing street values and the missing <i>location</i> value

In [13]:
# find the subset of the data where the street is null && the location is null
missing_loc = df.loc[(df['STREET'].isnull()) & (df['Location'].isnull() )]
# find the count of rows in this subset
print('number of overlapping missing values is: ', missing_loc['INCIDENT_NUMBER'].count())

number of overlapping missing values is:  10003


<b>The above means that we have <u>10,003</u> missing location and street information</b>

do the same for checking if the missing district overlaps with any of the street or location

In [14]:
# find the subset of the data where 
# the street is null && the location is null
# and the district is null
missing_dist = missing_loc.loc[(missing_loc['DISTRICT'].isnull())]
# find the count of rows in this subset
print('number of overlapping missing values is: ', missing_dist['INCIDENT_NUMBER'].count())

number of overlapping missing values is:  228


<b>the above calculation means that we have 228 rows where the street, district, and location are missing</b>. this means that we can go ahead and drop these rows as there is no way of determining the true location of the crimes associated with these entries.

it is necessary at this stage to define the UCR_PART column. by further examining the UCR_PART column: 

For reporting purposes, criminal offenses are divided into two major groups: Part I offenses and Part II offenses:<br>
(source: https://en.wikipedia.org/wiki/Uniform_Crime_Reports)

<b>Part I:</b> In Part I, the UCR indexes reported incidents of index crimes which are broken into two categories: violent and property crimes. they can be summarized as:  Aggravated assault, forcible rape, murder, and robbery are classified as violent while arson, burglary, larceny-theft, and motor vehicle theft are classified as property crimes

<b>Part II:</b> the following categories are tracked: simple assault, curfew offenses and loitering, embezzlement, forgery and counterfeiting, disorderly conduct, driving under the influence, drug offenses, fraud, gambling, liquor offenses, offenses against the family, prostitution, public drunkenness, runaways, sex offenses, stolen property, vandalism, vagrancy, and weapons offenses

<br>
the data include crimes that are referred to as <b>Part III</b>, <b>Other</b>, and <b>blank</b>, a following is a sample of these crimes (rows 200 to 210):

    


In [15]:
other_crimes = df.loc[(df['UCR_PART'] == 'Part Three')]
other_crimes.loc[200:210,]


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
200,I192078415,3006,Medical Assistance,SICK/INJURED/MEDICAL - PERSON,C11,332,N,2019-09-28 12:24:00,2019,9,Saturday,12,Part Three,COLUMBIA RD,42.313634,-71.067780,"(42.31363409, -71.06778025)"
201,I192078413,3803,Motor Vehicle Accident Response,M/V ACCIDENT - PERSONAL INJURY,B3,466,N,2019-09-28 12:24:00,2019,9,Saturday,12,Part Three,GOODALE RD,42.283628,-71.093496,"(42.28362830, -71.09349553)"
202,I192078411,3109,Police Service Incidents,SERVICE TO OTHER PD INSIDE OF MA.,B2,278,N,2019-09-28 12:33:00,2019,9,Saturday,12,Part Three,EUSTIS ST,42.329463,-71.079228,"(42.32946303, -71.07922822)"
203,I192078409,3503,Missing Person Located,MISSING PERSON - NOT REPORTED - LOCATED,D4,619,N,2019-09-28 12:42:00,2019,9,Saturday,12,Part Three,GAINSBOROUGH ST,42.342391,-71.088720,"(42.34239114, -71.08872018)"
204,I192078408,3201,Property Lost,PROPERTY - LOST,A1,128,N,2019-09-28 10:15:00,2019,9,Saturday,10,Part Three,ARLINGTON ST,42.348670,-71.069389,"(42.34866982, -71.06938919)"
206,I192078406,3802,Motor Vehicle Accident Response,M/V ACCIDENT - PROPERTY Â DAMAGE,E5,676,N,2019-09-28 12:19:00,2019,9,Saturday,12,Part Three,SOUTH ST,42.288025,-71.143697,"(42.28802461, -71.14369706)"
209,I192078402,3410,Towed,TOWED MOTOR VEHICLE,D4,624,N,2019-09-28 10:50:00,2019,9,Saturday,10,Part Three,JERSEY ST,NaN,NaN,NaN


In [16]:
#crimes with blank UCR_PART

BlankUCR_crimes = df.loc[(df['UCR_PART'].isnull())]
BlankUCR_crimes.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
6649,I192071414,1620,HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE,HUMAN TRAFFICKING - INVOLUNTARY SERVITUDE,B3,445,N,2019-09-06 18:18:00,2019,9,Friday,18,NaN,BRENTON ST,42.297605,-71.080249,"(42.29760546, -71.08024924)"
7745,I192070219,2010,HOME INVASION,HOME INVASION,E18,940,N,2019-09-03 11:10:00,2019,9,Tuesday,11,NaN,GARFIELD AVE,42.243619,-71.119893,"(42.24361883, -71.11989255)"
13523,I192063904,2010,HOME INVASION,HOME INVASION,E18,503,N,2019-08-14 22:00:00,2019,8,Wednesday,22,NaN,MOUNT HOPE ST,42.281964,-71.114240,"(42.28196390, -71.11424049)"
14385,I192062990,2010,HOME INVASION,HOME INVASION,B2,288,N,2019-08-11 18:20:00,2019,8,Sunday,18,NaN,MADISON PARK CT,42.331029,-71.086183,"(42.33102946, -71.08618278)"
23325,I192053622,2010,HOME INVASION,HOME INVASION,B3,467,N,2019-07-12 16:01:00,2019,7,Friday,16,NaN,GILMER ST,42.282811,-71.097223,"(42.28281074, -71.09722321)"


In [17]:
#see how many mussing 
print(BlankUCR_crimes['INCIDENT_NUMBER'].count())

110


In [18]:
#crimes with Other UCR_PART

BlankUCR_crimes = df.loc[(df['UCR_PART'] == 'Other')]
BlankUCR_crimes.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
799,I192077725,735,Auto Theft Recovery,RECOVERED - MV RECOVERED IN BOSTON (STOLEN OUT...,C11,392,N,2019-09-26 10:27:00,2019,9,Thursday,10,Other,ASHMONT ST,42.287346,-71.058187,"(42.28734575, -71.05818703)"
839,I192077681,735,Auto Theft Recovery,RECOVERED - MV RECOVERED IN BOSTON (STOLEN OUT...,C11,454,N,2019-09-26 08:40:00,2019,9,Thursday,8,Other,WASHINGTON ST,42.300226,-71.074683,"(42.30022551, -71.07468313)"
846,I192077674,735,Auto Theft Recovery,RECOVERED - MV RECOVERED IN BOSTON (STOLEN OUT...,D4,167,N,2019-09-25 19:00:00,2019,9,Wednesday,19,Other,E BROOKLINE ST,42.337635,-71.070130,"(42.33763461, -71.07013024)"
1406,I192077061,735,Auto Theft Recovery,RECOVERED - MV RECOVERED IN BOSTON (STOLEN OUT...,B2,317,N,2019-09-24 12:10:00,2019,9,Tuesday,12,Other,HOMESTEAD ST,42.311277,-71.089093,"(42.31127726, -71.08909334)"
1471,I192076991,735,Auto Theft Recovery,RECOVERED - MV RECOVERED IN BOSTON (STOLEN OUT...,C11,344,N,2019-09-24 08:47:00,2019,9,Tuesday,8,Other,CORONA ST,42.301462,-71.068960,"(42.30146198, -71.06895960)"


In [19]:
# how many is missing ?
print(BlankUCR_crimes['INCIDENT_NUMBER'].count())

1644


from the above sample we can notice the following crimes:
 <ol>
    <li><b>Part 3:</b>
        <ol>
            <li>Motor vehicle accident</li>
            <li>Service to other PD</li>
            <li>Missing person located</li>
        </ol></li>
    <li><b>Other:</b>
        <ol>
            <li>Auto theft recovery</li>
            <li>Recovered stolen plates</li>
            <li>Arson</li>
        </ol></li>
    <li><b>Blank:</b>
        <ol>
            <li>Human traficking</li>
            <li>Home Invasion</li>
        </ol></li>
    
 </ol>
 
 these crimes are indicative that part 3 and other crimes are either response incidents to previous crimes, or crimes that are committed at other locations and concluded at the location, when further invistigating the data, it was noticed that most of these crimes have missing locations implying that the crimes were not committed where they where reported and the original location is unknown. 
 
the following section will try to understand this further. 

In [20]:
# how many part 1 crimes have missing values?
part1_missing = df.loc[(df['UCR_PART'] == 'Part One') & (df['Location'].isnull() )]
no_part1_missing = part1_missing['INCIDENT_NUMBER'].count()
print('number of part 1 missing location is: ', no_part1_missing)

number of part 1 missing location is:  3558


In [21]:
# how many part 2 crimes have missing values?
part2_missing = df.loc[(df['UCR_PART'] == 'Part Two') & (df['Location'].isnull() )]
no_part2_missing = part2_missing['INCIDENT_NUMBER'].count()
print('number of part 2 missing location is: ', no_part2_missing)

number of part 2 missing location is:  8777


In [22]:
# how many part 3 crimes have missing values?
part3_missing = df.loc[(df['UCR_PART'] == 'Part Three') & (df['Location'].isnull() )]
no_part3_missing = part3_missing['INCIDENT_NUMBER'].count()
print('number of part 3 missing location is: ', no_part3_missing)

number of part 3 missing location is:  15680


In [23]:
# how many part other crimes have missing values?
partOther_missing = df.loc[(df['UCR_PART'] == 'Other') & (df['Location'].isnull() )]
no_partOther_missing = partOther_missing['INCIDENT_NUMBER'].count()
print('number of part Other missing location is: ', no_partOther_missing)

number of part Other missing location is:  115


In [24]:
# how many Blank UCR_PART crimes have missing values?
partBlank_missing = df.loc[(df['UCR_PART'].isnull()) & (df['Location'].isnull() )]
no_partBlank_missing = partBlank_missing['INCIDENT_NUMBER'].count()
print('number of Blank UCR_PART missing location is: ', no_partBlank_missing)

number of Blank UCR_PART missing location is:  1


In [25]:
#make sure that we accounted for all the missing data:
total_UCR_Missing = no_part1_missing+no_part2_missing+no_part3_missing+no_partOther_missing+no_partBlank_missing
print('number of part 1 missing location is: ', no_part1_missing)
print('number of part 2 missing location is: ', no_part2_missing)
print('number of part 3 missing location is: ', no_part3_missing)
print('number of part Other missing location is: ', no_partOther_missing)
print('number of Blank UCR_PART missing location is: ', no_partBlank_missing)
print('total = ', total_UCR_Missing)

number of part 1 missing location is:  3558
number of part 2 missing location is:  8777
number of part 3 missing location is:  15680
number of part Other missing location is:  115
number of Blank UCR_PART missing location is:  1
total =  28131


at this point it is noticed that most of the missing values are in part 3. and as part 3 is irrelevant to out prediction model and will be deleted, deleting it will remove most of the missing locations in the data. 

final check before deleting all data with missing locations it is important to know how many data in categories other that part 3 and other will be lost.
the following check will give the numbers

In [26]:
#group by UCR to see how many of each part is in the data
part_count = df.groupby('UCR_PART').count()
part_count

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,STREET,Lat,Long,Location
UCR_PART,,,,,,,,,,,,,,,,
Other,1644,1644,1644,1644,1630,1644,1644,1644,1644,1644,1644,1644,1597,1529,1529,1529
Part One,80783,80783,80783,80783,80539,80783,80783,80783,80783,80783,80783,80783,79228,77225,77225,77225
Part Three,214043,214043,214043,214043,212752,214043,214043,214043,214043,214043,214043,214043,207218,198363,198363,198363
Part Two,130260,130260,130260,130260,129640,130260,130260,130260,130260,130260,130260,130260,126296,121483,121483,121483


In [27]:
part1Count = part_count.iloc[1,0]
part2Count = part_count.iloc[3,0]
part3Count = part_count.iloc[2,0]
partOtherCount = part_count.iloc[0,0]
partBlankCount = 110
print('Part 1 count: ', part1Count)
print('Part 2 count: ', part2Count)
print('Part 3 count: ', part3Count)
print('Part Other count: ', partOtherCount)
print('Part Blank count: ', partBlankCount)
print('Total ', part1Count+part2Count+part3Count+
                  partOtherCount+partBlankCount)

Part 1 count:  80783
Part 2 count:  130260
Part 3 count:  214043
Part Other count:  1644
Part Blank count:  110
Total  426840


## Create a subset of the data
where UCR Part 3 and Other are ommitted and Part Blank is filled with the string 'Blank'

In [28]:
#make a copy of df
crime = df.copy()
# fill blank values in UCR_Part with 'Blank'
crime['UCR_PART'].fillna('Blank', inplace = True)

Part_One = crime[crime.UCR_PART == 'Part One']
Part_Two = crime[crime.UCR_PART == 'Part Two']
Part_Blank = crime[crime.UCR_PART == 'Blank']

data = pd.concat([Part_One, Part_Two, Part_Blank])
# remove rows with missing location values
data = data.dropna(subset=['Lat', 'Long', 'Location'])
# remove rows with missing district values
data = data.dropna(subset=['DISTRICT'])


Our data is now almost ready, final step is to delete the duplicates, and delete rows with null values

In [29]:
data = data.drop_duplicates()
#save the final dataset to file
# data.to_csv('data.csv')

In [30]:
# make sure we dont have missing values 
data.isnull().sum()

INCIDENT_NUMBER          0
OFFENSE_CODE             0
OFFENSE_CODE_GROUP       0
OFFENSE_DESCRIPTION      0
DISTRICT                 0
REPORTING_AREA           0
SHOOTING                 0
OCCURRED_ON_DATE         0
YEAR                     0
MONTH                    0
DAY_OF_WEEK              0
HOUR                     0
UCR_PART                 0
STREET                 606
Lat                      0
Long                     0
Location                 0
dtype: int64

it is noticed that 606 missing street values are remaining so we need to delete those as well because it will be difficult to populate them 

In [31]:
# we have 606 missing street so we will delete these rows
data = data.dropna(subset=['STREET'])
#check agains
data.isnull().sum()

INCIDENT_NUMBER        0
OFFENSE_CODE           0
OFFENSE_CODE_GROUP     0
OFFENSE_DESCRIPTION    0
DISTRICT               0
REPORTING_AREA         0
SHOOTING               0
OCCURRED_ON_DATE       0
YEAR                   0
MONTH                  0
DAY_OF_WEEK            0
HOUR                   0
UCR_PART               0
STREET                 0
Lat                    0
Long                   0
Location               0
dtype: int64

#### Final step of data cleaning:
at this point the data is clean from missing values, next we need to:
1. remove unwanted or redundant columns 
2. create new columns that are useful for analysis

A. Extract Date from occured on Date column

In [32]:
#split occured_on_date column to remove time
date = data['OCCURRED_ON_DATE'].str.split(" ", n = 2, expand = True)

In [41]:
date.head()

,Date,Time
0,2019-10-13,09:28:24
1,2019-10-12,20:11:26
2,2019-10-12,15:12:43
3,2019-10-12,04:41:52
4,2019-10-02,08:08:49


In [33]:
#create a new column Date for each day of data
date.columns = ['Date', 'Time']
data['Date'] = date['Date']

In [34]:
data.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location,Date
0,I192082859,724,Auto Theft,AUTO THEFT,E18,519,N,2019-10-13 09:28:24,2019,10,Sunday,9,Part One,LINCOLN ST,42.259518,-71.121563,"(42.25951765, -71.12156299)",2019-10-13
1,I192082751,724,Auto Theft,AUTO THEFT,E18,493,N,2019-10-12 20:11:26,2019,10,Saturday,20,Part One,METROPOLITAN AVE,42.262092,-71.116710,"(42.26209214, -71.11670964)",2019-10-12
2,I192082680,727,Auto Theft,AUTO THEFT - LEASED/RENTED VEHICLE,D14,794,N,2019-10-12 15:12:43,2019,10,Saturday,15,Part One,ALLSTON ST,42.352375,-71.135096,"(42.35237455, -71.13509584)",2019-10-12
3,I192082577,724,Auto Theft,AUTO THEFT,D4,130,N,2019-10-12 04:41:52,2019,10,Saturday,4,Part One,SAINT JAMES AVE,42.349476,-71.076402,"(42.34947586, -71.07640150)",2019-10-12
4,I192079582,727,Auto Theft,AUTO THEFT - LEASED/RENTED VEHICLE,A15,47,N,2019-10-02 08:08:49,2019,10,Wednesday,8,Part One,N MEAD ST,42.381846,-71.066551,"(42.38184582, -71.06655134)",2019-10-02


In [42]:
# extract month and day from the new date column
MONTH = data['Date'].str.split("-", n = 2, expand = True)

In [44]:
MONTH.head()

,0,1,2
0,2019,10,13
1,2019,10,12
2,2019,10,12
3,2019,10,12
4,2019,10,02


In [45]:


#add new columns Month and Day to the data
data['MONTH'] = MONTH[1]
data['DAY'] = MONTH[2]

2. Remove unwanted columns 

In [46]:
# drop unwanted columns
data.drop(columns=['INCIDENT_NUMBER', 'OFFENSE_DESCRIPTION', 'Location', 'OCCURRED_ON_DATE', 'Date'], inplace=True)

In [47]:
data.head()

,OFFENSE_CODE,OFFENSE_CODE_GROUP,DISTRICT,REPORTING_AREA,SHOOTING,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,DAY
0,724,Auto Theft,E18,519,N,2019,10,Sunday,9,Part One,LINCOLN ST,42.259518,-71.121563,13
1,724,Auto Theft,E18,493,N,2019,10,Saturday,20,Part One,METROPOLITAN AVE,42.262092,-71.116710,12
2,727,Auto Theft,D14,794,N,2019,10,Saturday,15,Part One,ALLSTON ST,42.352375,-71.135096,12
3,724,Auto Theft,D4,130,N,2019,10,Saturday,4,Part One,SAINT JAMES AVE,42.349476,-71.076402,12
4,727,Auto Theft,A15,47,N,2019,10,Wednesday,8,Part One,N MEAD ST,42.381846,-71.066551,02


### Last thing is to change numerical values stored as strings to int

In [48]:
# function to check column data types 
def printColumnInfo(df):
    template="%-8s %-30s %s"
    print(template % ("Type", "Column Name", "Example Value"))
    print("-"*53)
    for c in df.columns:
        print(template % (df[c].dtype, c, df[c].iloc[1]) )

In [49]:
printColumnInfo(data)

Type     Column Name                    Example Value
-----------------------------------------------------
int64    OFFENSE_CODE                   724
object   OFFENSE_CODE_GROUP             Auto Theft
object   DISTRICT                       E18
object   REPORTING_AREA                 493
object   SHOOTING                       N
int64    YEAR                           2019
object   MONTH                          10
object   DAY_OF_WEEK                    Saturday
int64    HOUR                           20
object   UCR_PART                       Part One
object   STREET                         METROPOLITAN AVE
float64  Lat                            42.26209214
float64  Long                           -71.11670964
object   DAY                            12


change columns of numeric stored as string types to float:
    1. reporting area 
    2. Month
    3. Day
    
#### 1. Reporting Area 


In [50]:
# find the number of unique values
no_area = data['REPORTING_AREA'].unique().size
no_area

878

In [51]:
# find the unique values
area = data['REPORTING_AREA'].unique()
# area

In [52]:
# create a new code values
new_area = np.arange(0, no_area, 1)
# new_area

In [53]:
# replace values accordiing to the new codes
for i in new_area:
    data.loc[(data.REPORTING_AREA == area[i]),'REPORTING_AREA']= i

In [54]:
# df
#check if the replacement is successful
# data['REPORTING_AREA'].unique()
data.head(15)

,OFFENSE_CODE,OFFENSE_CODE_GROUP,DISTRICT,REPORTING_AREA,SHOOTING,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,DAY
0,724,Auto Theft,E18,0,N,2019,10,Sunday,9,Part One,LINCOLN ST,42.259518,-71.121563,13
1,724,Auto Theft,E18,1,N,2019,10,Saturday,20,Part One,METROPOLITAN AVE,42.262092,-71.116710,12
2,727,Auto Theft,D14,2,N,2019,10,Saturday,15,Part One,ALLSTON ST,42.352375,-71.135096,12
3,724,Auto Theft,D4,3,N,2019,10,Saturday,4,Part One,SAINT JAMES AVE,42.349476,-71.076402,12
4,727,Auto Theft,A15,4,N,2019,10,Wednesday,8,Part One,N MEAD ST,42.381846,-71.066551,02
12,423,Aggravated Assault,A1,5,N,2019,09,Sunday,4,Part One,CAUSEWAY ST,42.364331,-71.063193,29
14,521,Residential Burglary,B2,6,N,2019,09,Sunday,4,Part One,ABBOTSFORD ST,42.314630,-71.092615,29
21,619,Larceny,C6,7,N,2019,09,Sunday,3,Part One,W BROADWAY,42.340070,-71.052794,29
25,423,Aggravated Assault,A7,8,N,2019,09,Sunday,2,Part One,CHELSEA ST,42.379124,-71.028082,29
30,301,Robbery,A1,9,N,2019,09,Sunday,1,Part One,TREMONT ST,42.356375,-71.062135,29


#### 2. Month

In [55]:
# find the number of unique values
no_month = data['MONTH'].unique().size
no_month

12

In [56]:
# find the unique values
# month = df['MONTH'].unique()
#define months in number order
month = [1,2,3,4,5,6,7,8,9,10,11,12]
month

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [57]:
# replace values accordiing to the new codes
data.loc[(data.MONTH == '1'),'MONTH']= 1
data.loc[(data.MONTH == '2'),'MONTH']= 2
data.loc[(data.MONTH == '3'),'MONTH']= 3
data.loc[(data.MONTH == '4'),'MONTH']= 4
data.loc[(data.MONTH == '5'),'MONTH']= 5
data.loc[(data.MONTH == '6'),'MONTH']= 6
data.loc[(data.MONTH == '7'),'MONTH']= 7
data.loc[(data.MONTH == '8'),'MONTH']= 8
data.loc[(data.MONTH == '9'),'MONTH']= 9
data.loc[(data.MONTH == '10'),'MONTH']= 10
data.loc[(data.MONTH == '11'),'MONTH']= 11
data.loc[(data.MONTH == '12'),'MONTH']= 12

data.head()

,OFFENSE_CODE,OFFENSE_CODE_GROUP,DISTRICT,REPORTING_AREA,SHOOTING,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,DAY
0,724,Auto Theft,E18,0,N,2019,10,Sunday,9,Part One,LINCOLN ST,42.259518,-71.121563,13
1,724,Auto Theft,E18,1,N,2019,10,Saturday,20,Part One,METROPOLITAN AVE,42.262092,-71.116710,12
2,727,Auto Theft,D14,2,N,2019,10,Saturday,15,Part One,ALLSTON ST,42.352375,-71.135096,12
3,724,Auto Theft,D4,3,N,2019,10,Saturday,4,Part One,SAINT JAMES AVE,42.349476,-71.076402,12
4,727,Auto Theft,A15,4,N,2019,10,Wednesday,8,Part One,N MEAD ST,42.381846,-71.066551,02


#### 3. Day

In [58]:
# find the number of unique values
no_days = data['DAY'].unique().size
no_days

31

In [59]:
# find the unique values
days = data['DAY'].unique()
days
#define months in number order
# month = [1,2,3,4,5,6,7,8,9,10,11,12]
# month

array(['13', '12', '02', '29', '28', '18', '27', '23', '26', '25', '16',
       '10', '24', '08', '03', '22', '14', '20', '19', '21', '15', '06',
       '11', '01', '30', '17', '07', '09', '04', '05', '31'], dtype=object)

In [60]:
# replace values accordiing to the new codes
data.loc[(data.DAY == '1'),'DAY']= 1
data.loc[(data.DAY == '2'),'DAY']= 2
data.loc[(data.DAY == '3'),'DAY']= 3
data.loc[(data.DAY == '4'),'DAY']= 4
data.loc[(data.DAY == '5'),'DAY']= 5
data.loc[(data.DAY == '6'),'DAY']= 6
data.loc[(data.DAY == '7'),'DAY']= 7
data.loc[(data.DAY == '8'),'DAY']= 8
data.loc[(data.DAY == '9'),'DAY']= 9
data.loc[(data.DAY == '10'),'DAY']= 10
data.loc[(data.DAY == '11'),'DAY']= 11
data.loc[(data.DAY == '12'),'DAY']= 12
data.loc[(data.DAY == '13'),'DAY']= 13
data.loc[(data.DAY == '14'),'DAY']= 14
data.loc[(data.DAY == '15'),'DAY']= 15
data.loc[(data.DAY == '16'),'DAY']= 16
data.loc[(data.DAY == '17'),'DAY']= 17
data.loc[(data.DAY == '18'),'DAY']= 18
data.loc[(data.DAY == '19'),'DAY']= 19
data.loc[(data.DAY == '20'),'DAY']= 20
data.loc[(data.DAY == '21'),'DAY']= 21
data.loc[(data.DAY == '22'),'DAY']= 22
data.loc[(data.DAY == '23'),'DAY']= 23
data.loc[(data.DAY == '24'),'DAY']= 24
data.loc[(data.DAY == '25'),'DAY']= 25
data.loc[(data.DAY == '26'),'DAY']= 26
data.loc[(data.DAY == '27'),'DAY']= 27
data.loc[(data.DAY == '28'),'DAY']= 28
data.loc[(data.DAY == '29'),'DAY']= 29
data.loc[(data.DAY == '30'),'DAY']= 30
data.loc[(data.DAY == '31'),'DAY']= 31

data.head()

,OFFENSE_CODE,OFFENSE_CODE_GROUP,DISTRICT,REPORTING_AREA,SHOOTING,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,DAY
0,724,Auto Theft,E18,0,N,2019,10,Sunday,9,Part One,LINCOLN ST,42.259518,-71.121563,13
1,724,Auto Theft,E18,1,N,2019,10,Saturday,20,Part One,METROPOLITAN AVE,42.262092,-71.116710,12
2,727,Auto Theft,D14,2,N,2019,10,Saturday,15,Part One,ALLSTON ST,42.352375,-71.135096,12
3,724,Auto Theft,D4,3,N,2019,10,Saturday,4,Part One,SAINT JAMES AVE,42.349476,-71.076402,12
4,727,Auto Theft,A15,4,N,2019,10,Wednesday,8,Part One,N MEAD ST,42.381846,-71.066551,02


In [61]:
#one final check
printColumnInfo(data)

Type     Column Name                    Example Value
-----------------------------------------------------
int64    OFFENSE_CODE                   724
object   OFFENSE_CODE_GROUP             Auto Theft
object   DISTRICT                       E18
int64    REPORTING_AREA                 1
object   SHOOTING                       N
int64    YEAR                           2019
object   MONTH                          10
object   DAY_OF_WEEK                    Saturday
int64    HOUR                           20
object   UCR_PART                       Part One
object   STREET                         METROPOLITAN AVE
float64  Lat                            42.26209214
float64  Long                           -71.11670964
object   DAY                            12


In [62]:
#save data to file
data.to_csv('cleanBoston.csv', index=False)